# Hanukkah Of Data 5783

> Shut up and calculate!

-- David Mermin

## Imports and Definitions

In [7]:
import pandas as pd
from operator import or_
from functools import reduce
from itertools import zip_longest
import urllib.request
from zipfile import ZipFile

## Downloading Data

In [8]:
url = 'https://hanukkah.bluebird.sh/5783/noahs-csv.zip'
filename = "noahs-csv.zip"
password = b"5777"

with (
    urllib.request.urlopen(url) as f,
    open(filename, "wb") as o,
):
    o.write(f.read())

with ZipFile(filename) as z:
    z.extractall(path='./data', pwd=password)

## [Puzzle 1](https://hanukkah.bluebird.sh/5783/1/)

In [9]:
def is_matching(s, phone):
    l = {
        2: "abc", 3: "def", 4: "ghi",
        5: "jkl", 6: "mno", 7: "pqrs",
        8: "tuv", 9: "wxyz"
    }
    m = reduce(or_, ({c: k for c in v} for k, v in l.items()))
    
    n = (c.lower() for c in s if c.isalpha())
    p = (int(c) for c in phone if c.isdigit())
    return all(m.get(a, '?') == b for a, b in zip_longest(n, p))


def last_name(name):
    return name.split()[1]


df = pd.read_csv('data/noahs-customers.csv')
df['last_name'] = df.apply(lambda r: last_name(r['name']), axis=1)
df['match'] = df.apply(lambda r: is_matching(r['last_name'], r['phone']), axis=1)
df.query('match')

,customerid,name,address,citystatezip,birthdate,phone,last_name,match
2187,3188,Sam Guttenberg,221 Banker St,"Brooklyn, NY 11222",1998-05-30,488-836-2374,Guttenberg,True


## [Puzzle 2](https://hanukkah.bluebird.sh/5783/2/)

In [10]:
def initials(name):
    return ''.join(s[0] for s in name.split())
    

c = pd.read_csv('data/noahs-customers.csv').set_index('customerid')
o = pd.read_csv('data/noahs-orders.csv').set_index('customerid')


j = (
    o.join(c)
     .groupby('customerid')
     .max()
)

j['initials'] = j.apply(lambda r: initials(r['name']), axis=1)
j.query('ordered < "2018-01-01" and initials == "JD"')

,orderid,ordered,shipped,items,total,name,address,citystatezip,birthdate,phone,initials
customerid,,,,,,,,,,,
3704,25377,2017-10-03 13:21:53,2017-10-04 11:15:00,NaN,19.67,James Davis,2380 Park Dr,"Dallas, TX 75227",1974-04-08,682-969-2705,JD
4164,8557,2017-04-17 14:39:59,2017-04-17 14:39:59,NaN,33.22,Jeremy Davis,134-10 Foch Blvd,"South Ozone Park, NY 11420",1995-07-26,212-771-8924,JD


## [Puzzle 3](https://hanukkah.bluebird.sh/5783/3/)

> Aries (♈︎) (Greek: Κριός, romanized: Kriós, Latin for "ram") is the first astrological sign in the zodiac, spanning the first 30 degrees of celestial longitude (0°≤ λ <30°), and originates from the Aries constellation. Under the tropical zodiac, the Sun transits this sign from approximately **March 21 to April 19** each year.

> The sexagenary cycle, also known as the Stems-and-Branches or ganzhi (Chinese: 干支), is a cycle of sixty terms, each corresponding to one year, thus a total of sixty years for one cycle, historically used for recording time in China and the rest of the East Asian cultural sphere.

> Each term in the sexagenary cycle consists of two Chinese characters, the first being one of the ten Heavenly Stems of the Shang-era week and the second being one of the twelve Earthly Branches representing the years of Jupiter's duodecennial orbital cycle. The first term jiǎzǐ (甲子) combines the first heavenly stem with the first earthly branch. The second term yǐchǒu (乙丑) combines the second stem with the second branch.

I guess this explains why it's called the *chinese* remainder theorem.

Dog years are congruent to $2$ $\pmod {12}$.

In [11]:
def is_dog(date):
    return date.year % 12 == 2


def is_aries(date):
    return (
        (date.month == 3 and date.day >= 21) or 
        (date.month == 4 and date.day <= 19)
    )


df = pd.read_csv('data/noahs-customers.csv')

df['birthdate'] = pd.to_datetime(df['birthdate'])    
df['is_dog'] = df.apply(lambda r: is_dog(r['birthdate']), axis=1)
df['is_aries'] = df.apply(lambda r: is_aries(r['birthdate']), axis=1)

df.query('is_dog and is_aries and citystatezip == "South Ozone Park, NY 11420"')

,customerid,name,address,citystatezip,birthdate,phone,is_dog,is_aries
1273,2274,Brent Nguyen,109-19 110th St,"South Ozone Park, NY 11420",1958-03-25,516-636-7397,True,True


## [Puzzle 4](https://hanukkah.bluebird.sh/5783/4/)

In [12]:
p = pd.read_csv('data/noahs-products.csv')
o = pd.read_csv('data/noahs-orders.csv').set_index('orderid')
c = pd.read_csv('data/noahs-customers.csv').set_index('customerid')
op = pd.read_csv('data/noahs-orders_items.csv').set_index('orderid')


def is_dawn(date):
    return date.hour == 4


p['is_pastry'] = p.apply(lambda r: r['sku'].startswith('BKY'), axis=1)
p = p.query('is_pastry').set_index('sku')

j = (
    op.join(o)
        .join(p, on='sku', how='inner')
        .join(c, on='customerid')
)

j['shipped'] = pd.to_datetime(j['shipped'])
j['ordered'] = pd.to_datetime(j['ordered'])
j['is_dawn'] = j.apply(lambda r: is_dawn(r['shipped']) and is_dawn(r['ordered']), axis=1)
j.query('is_dawn')[['name', 'phone']].drop_duplicates()


,name,phone
orderid,,
201780,Christina Booker,718-649-9036
26173,Heather Brown,929-518-7269


## [Puzzle 5](https://hanukkah.bluebird.sh/5783/5/)

In [13]:
p = pd.read_csv('data/noahs-products.csv').set_index('sku')
o = pd.read_csv('data/noahs-orders.csv').set_index('orderid')
c = pd.read_csv('data/noahs-customers.csv').set_index('customerid')
op = pd.read_csv('data/noahs-orders_items.csv').set_index('orderid')

j = (
    op.join(o)
        .join(p, on='sku', how='inner')
        .join(c, on='customerid')
)

j['is_petfood'] = j.apply(lambda r: r['sku'].startswith('PET'), axis=1)

agg = (
    j[['customerid', 'is_petfood']]
        .groupby('customerid')
        .sum()
        .query('is_petfood > 0')
)

agg.join(c, on='customerid') \
    .query('citystatezip == "Queens Village, NY 11429"') \
    .sort_values('is_petfood', ascending=False)

,is_petfood,name,address,citystatezip,birthdate,phone
customerid,,,,,,
7675,17,Anita Koch,106-51 214th St,"Queens Village, NY 11429",1955-11-14,315-492-7411
5486,3,Sam House,220-8 111th Ave,"Queens Village, NY 11429",1955-11-10,607-836-2966
3335,1,Jose Cook,225-24 Murdock Ave,"Queens Village, NY 11429",1950-03-27,516-745-9461


## [Puzzle 6](https://hanukkah.bluebird.sh/5783/6/)

In [14]:
p = pd.read_csv('data/noahs-products.csv').set_index('sku')
o = pd.read_csv('data/noahs-orders.csv').set_index('orderid')
c = pd.read_csv('data/noahs-customers.csv').set_index('customerid')
op = pd.read_csv('data/noahs-orders_items.csv').set_index('orderid')

j = op.join(p, on='sku')
j['diff'] = j.unit_price - j.wholesale_cost
j = j[['diff']].groupby('orderid').sum().query('diff < 0')

g = j.join(o).groupby('customerid').count()
g.join(c).sort_values('total', ascending=False)

,diff,ordered,shipped,items,total,name,address,citystatezip,birthdate,phone
customerid,,,,,,,,,,
8342,15,15,15,0,15,Emily Randolph,1055A E 3rd St,"Brooklyn, NY 11230",1988-10-30,914-868-0316
9723,3,3,3,0,3,Matthew Holmes,186½ W 105th St,"Manhattan, NY 10025",1950-03-02,516-986-1842
9893,3,3,3,0,3,Robert White,1503 Leland Ave,"Bronx, NY 10460",1959-11-26,914-548-7137
2870,3,3,3,0,3,Steven Reilly,969 Lexington Ave,"Manhattan, NY 10021",1995-09-26,680-315-1072
2956,3,3,3,0,3,Jonathan Williams,2116B 3rd Ave,"Manhattan, NY 10029",1943-08-13,332-668-0158
...,...,...,...,...,...,...,...,...,...,...
5173,1,1,1,0,1,John Long,713C E Tremont Ave,"Bronx, NY 10457",1970-08-14,680-421-2701
5231,1,1,1,0,1,Maria Cross,20-79 Crescent St,"Astoria, NY 11105",1969-09-17,680-413-4429
5264,1,1,1,0,1,Melissa Munoz,241A E 204th St,"Bronx, NY 10458",1983-12-05,516-377-0946


## [Puzzle 7](https://hanukkah.bluebird.sh/5783/7/)

In [15]:
op = pd.read_csv('data/noahs-orders_items.csv').set_index('orderid')
p = pd.read_csv('data/noahs-products.csv').set_index('sku')
o = pd.read_csv('data/noahs-orders.csv').set_index('orderid')
c = pd.read_csv('data/noahs-customers.csv').set_index('customerid')


def item_type_color(str):
    l = str.split()
    if l[-1].startswith('('):
        return ' '.join(l[:-1]), l[-1]
    else:
        return ' '.join(l), ''
    

j = op.join(p, on='sku')
j['type'] = j.apply(lambda r: item_type_color(r['desc'])[0], axis=1)
j['color'] = j.apply(lambda r: item_type_color(r['desc'])[1], axis=1)
j = j.query('color != ""')

g = j.join(o, on='orderid')

g['ordered'] = g.apply(
    lambda r: ''.join(r['ordered'].replace(':', ' ').split()[0:3]), axis=1
)

g = (
    g.merge(g, how='inner', on=['ordered', 'type'])
        .query('color_x != color_y')
        .query('customerid_x == 8342')
)

g.join(c, on='customerid_y')[['name', 'phone']]

,name,phone
28271,Jonathan Adams,315-618-5263


## [Puzzle 8](https://hanukkah.bluebird.sh/5783/8/)

In [16]:
op = pd.read_csv('data/noahs-orders_items.csv').set_index('orderid')
o = pd.read_csv('data/noahs-orders.csv').set_index('orderid')
c = pd.read_csv('data/noahs-customers.csv').set_index('customerid')


j = op.join(o)
j['is_collectible'] = j.apply(lambda r: r['sku'].startswith('COL'), axis=1)

cid = (
    j.query('is_collectible')
        .groupby('customerid')[['sku']]
        .nunique()
)

cid.join(c).sort_values('sku', ascending=False)

,sku,name,address,citystatezip,birthdate,phone
customerid,,,,,,
4308,81,Travis Bartlett,2527B Adam Clayton Powell Jr Blvd,"Manhattan, NY 10039",1942-07-22,929-906-5980
4308,81,Travis Bartlett,2527B Adam Clayton Powell Jr Blvd,"Manhattan, NY 10039",1942-07-22,929-906-5980
3076,29,Justin Jimenez,102½ E 84th St,"Manhattan, NY 10028",1989-03-16,516-722-4758
5313,27,Joshua Smith,97 W 111th St,"Manhattan, NY 10026",1986-06-04,914-243-3254
1228,26,Christopher Bennett,859 E 133rd St,"Bronx, NY 10454",1944-05-03,516-686-9853
...,...,...,...,...,...,...
7912,1,Kevin Williams,23-45 White Oak Ct,"East Elmhurst, NY 11370",1975-01-08,838-946-9841
2552,1,Anna Myers,620 1st Way NW,"Dover, NH 03820",1958-07-13,603-696-9093
7897,1,Felicia Brewer,3292 SW Main Dr,"Cedar City, UT 84721",1991-11-23,435-322-8020
